In [2]:
# Import modules

from music21 import *
import matplotlib.pyplot as plt
import pickle
import dimod.generators
import networkx as nx
import pandas as pd
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.samplers import SteepestDescentSolver
import dimod

from calculators import *
from boundaryDetection import *
from graphUtilities import *

In [3]:
# https://www.music21.org/music21docs/about/referenceCorpus.html#referencecorpus

beethoven = corpus.parse("beethoven/opus74.mxl")
excerpt = beethoven
identifier = "opus74"

#excerpt.show("text")

In [4]:
# Boundary detection

phraseStack = []
threshold = 0.15

for part in excerpt.parts:
    boundaries = identifyBoundaries(part, threshold)
    print(boundaries)
    phrases = extractPhrases(part, boundaries, f"{part.id}")    
    phraseStack.append(phrases)

[36.0, 55.75, 68.0, 100.0, 106.0, 141.5, 154.0, 166.0, 224.0, 225.0, 252.5, 258.0, 259.5, 275.0, 312.0, 322.0, 327.0, 334.0, 340.0, 364.0, 393.5, 396.0, 413.5, 421.5, 433.5, 436.0, 437.5, 440.0, 441.5, 444.0, 449.5, 452.0, 453.5, 456.0, 457.5, 460.0, 461.5, 464.0, 465.5, 468.0, 469.5, 472.0, 493.5, 546.0, 551.0, 551.5, 556.0, 562.0, 600.0, 608.0, 626.0, 638.0, 658.0, 670.0, 676.0, 678.0, 684.0, 728.0, 729.0, 743.0, 959.75, 960.0, 960.25, 967.75, 968.0, 968.25, 975.75, 976.0, 980.0, 982.5, 1032.0, 1034.0, 1038.0, 1049.5, 1235.5, 1260.0, 1263.5, 1303.0, 1306.0, 1310.0, 1396.0, 1400.0, 1401.0, 1403.0, 1404.0, 1406.0, 1407.0, 1409.0, 1410.0, 1411.0, 1414.0, 1415.5, 1418.0, 1419.0, 1421.0, 1422.0, 1424.0, 1425.0, 1427.0, 1428.0, 1531.0, 1589.5, 1664.5, 1667.5, 1697.5, 1703.5, 1715.5, 1811.5, 1814.5, 1818.5, 1904.5, 1908.5, 1909.5, 1911.5, 1912.5, 1914.5, 1915.5, 1917.5, 1918.5, 1919.5, 1922.5, 1924.0, 1926.5, 1927.5, 1929.5, 1930.5, 1932.5, 1933.5, 1935.5, 1936.5, 2039.0, 2096.5, 2171.5, 21

In [ ]:
# Create graph

flatStack = [phrase for phrases in phraseStack for phrase in phrases]
uniquePairs = [(i, j) for i in range(len(flatStack)) for j in range(i+1, len(flatStack))]

#print(len(flatStack))
#for p in flatStack: print(p.id, p.entropy)
#for i, j in uniquePairs: print(flatStack[i].id, flatStack[j].id, hasOverlap(flatStack[i], flatStack[j]))

edges = []

for i, j in uniquePairs:
    if hasOverlap(flatStack[i], flatStack[j]):
        edges.append((flatStack[i], flatStack[j]))


SimpleG = nx.Graph()
ObjectG = nx.Graph()

ObjectG.add_nodes_from(flatStack)
ObjectG.add_edges_from(edges)

"""
for p in flatStack:
    SimpleG.add_node(p.id)
    ObjectG.add_node(p)

    for n in ObjectG.nodes:
        if p != n and hasOverlap(p, n):
            SimpleG.add_edge(p.id, n.id)
            ObjectG.add_edge(p, n)

#G = defineGraph(edges)
"""
pos = nx.spring_layout(ObjectG, k=0.5) # Force specific layout
nx.draw_networkx(ObjectG, pos=pos, with_labels=False)

print(len(edges))

In [ ]:
phraseEntropies = [(phrase.id, phrase.entropy + 1) for phrase in flatStack]
bqm = dimod.generators.maximum_weight_independent_set(SimpleG.edges, phraseEntropies, strength_multiplier=2)
print(bqm)

In [ ]:
# Solve on QPU

def solveWeightedMIS(G: nx.Graph, weights: list[tuple] = [], sampler=EmbeddingComposite(DWaveSampler()), num_reads: int = 100, strength_multiplier: float = 2) -> pd.DataFrame:
    bqm = dimod.generators.maximum_weight_independent_set(G.edges, weights, strength_multiplier=strength_multiplier)
    sampler = sampler
    sampleset = sampler.sample(bqm, num_reads = num_reads)
    return sampleset.to_pandas_dataframe(True)

df = solveWeightedMIS(SimpleG, phraseEntropies, num_reads=1000)

In [ ]:
# Show results

print(df.head())
#print(df["energy"].sort_values(ascending=True))

sample = df["sample"][0]
chosen = [x for x in sample if sample[x] == 1]

print(chosen)
print(len(chosen))

nx.draw_networkx(SimpleG, pos=pos, nodelist=chosen)

In [ ]:
# Final arrangement

arrangement = stream.Stream()

for phrase in flatStack:
    if phrase.id in chosen:
        arrangement.mergeElements(phrase)

arrangement.show()


In [38]:
# Save data

excerpt.write("xml", f"../Pickles/{identifier}_excerpt.xml")
pickle.dump(phraseStack, open(f"../Pickles/{identifier}_phraseStack.pkl", "wb"))
pickle.dump(SimpleG, open(f"../Pickles/{identifier}_simpleGraph.pkl", "wb"))
pickle.dump(ObjectG, open(f"../Pickles/{identifier}_objectGraph.pkl", "wb"))
pickle.dump(df, open(f"../Pickles/{identifier}_df.pkl", "wb"))